<a href="https://colab.research.google.com/github/SoudeepGhoshal/ColabFiltering/blob/main/PIP_CollaborativeFiltering_MovieLens1M_Imputed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import pairwise_distances
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
cols_u = ['user_id', 'sex', 'age', 'occupation', 'zip_code']
users=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/users.dat', sep='::', names=cols_u, encoding='latin-1', parse_dates=True)

ratings_u = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/ratings.dat', sep='::', names=ratings_u, encoding='latin-1')

movies_u = ['movie_id' , 'title', 'genre']
movies = pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/movies.dat', sep='::', names=movies_u, encoding='latin-1')

movie_ratings=pd.merge(movies,ratings)
movie_df=pd.merge(movie_ratings,users)

movie_df.head(500)

<ipython-input-37-3433c4bfd01f>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/users.dat', sep='::', names=cols_u, encoding='latin-1', parse_dates=True)
<ipython-input-37-3433c4bfd01f>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/ratings.dat', sep='::', names=ratings_u, encoding='latin-1')
<ipython-input-37-3433c4bfd01f>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interp

,movie_id,title,genre,user_id,rating,timestamp,sex,age,occupation,zip_code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
495,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,10,5,979167660,F,35,1,95370
496,1198,Raiders of the Lost Ark (1981),Action|Adventure,10,5,978225630,F,35,1,95370
497,1200,Aliens (1986),Action|Sci-Fi|Thriller|War,10,5,979168160,F,35,1,95370
498,1201,"Good, The Bad and The Ugly, The (1966)",Action|Western,10,2,978225853,F,35,1,95370


In [38]:
#only for the normalization
min_rating = ratings['rating'].min()
max_rating = ratings['rating'].max()

ratings['rating'] = (ratings['rating'] - min_rating) / (max_rating - min_rating)

ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,1.00,978300760
1,1,661,0.50,978302109
2,1,914,0.50,978301968
3,1,3408,0.75,978300275
4,1,2355,1.00,978824291


In [39]:
movie_df.drop(movie_df.columns[[2,5]], axis=1, inplace=True)
ratings.drop("timestamp", inplace=True, axis=1)
movies.drop(movies.columns[[2]], inplace=True, axis=1)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   movie_id    1000209 non-null  int64 
 1   title       1000209 non-null  object
 2   user_id     1000209 non-null  int64 
 3   rating      1000209 non-null  int64 
 4   sex         1000209 non-null  object
 5   age         1000209 non-null  int64 
 6   occupation  1000209 non-null  int64 
 7   zip_code    1000209 non-null  object
dtypes: int64(5), object(3)
memory usage: 68.7+ MB


Run if not using Imputation on ratings

In [5]:
user_input_percent = 0.3
subset_percent = 0.7

num_ratings = len(ratings)
num_user_input = int(num_ratings * user_input_percent)
num_subset = int(num_ratings * subset_percent)

user_input_ratings = ratings.sample(n=num_user_input, random_state=42)
subset_ratings = ratings.drop(user_input_ratings.index)
user_input_ratings

,user_id,movie_id,rating
895536,5412,2683,0.25
899739,5440,904,1.00
55687,368,3717,0.75
63727,425,1721,0.75
822011,4942,3697,0.00
...,...,...,...
85492,558,923,1.00
586763,3582,329,0.75
589709,3597,3175,0.50
548798,3389,2987,0.75


Run for Imputation

In [41]:
userItemMatrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating')
userItemMatrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,0.25,NaN,0.50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
null_values = userItemMatrix.isnull().sum().sum()
print(f"Total number of null values: {null_values}")

Total number of null values: 21384031


In [43]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Create a copy of the userItemMatrix
userItemMatrix_imputed = userItemMatrix.copy()

# Calculate the total number of null values in the table
total_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the threshold for the number of null values to be imputed
null_threshold = int(total_null_values * 0.60)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# Iterate over each row
for index, row in userItemMatrix_imputed.iterrows():
    # Calculate the number of null values in each row
    row_null_values = row.isnull().sum()

    # If the number of null values in the row is less than the threshold, skip imputation for this row
    if row_null_values == 0 or row_null_values > null_threshold:
        continue

    # Create a mask for 60% of the null values in this row
    row_mask = np.random.choice([True, False], len(row), p=[0.6, 0.4])

    # Apply imputation for the selected 60% of null values in this row
    row_values = row.values
    row_values[row_mask] = imputer.fit_transform(row_values[row_mask].reshape(-1, 1)).ravel()

    # Update the values in the DataFrame for this row
    userItemMatrix_imputed.loc[index] = row_values

# Calculate the number of null values after imputation
remaining_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the number of null values that have been imputed
imputed_values = total_null_values - remaining_null_values

# Calculate the percentage of null values imputed
percent_imputed = (imputed_values / total_null_values) * 100

print(f"Percentage of null values imputed: {percent_imputed:.2f}%")
print(f"Remaining null values: {remaining_null_values}")

Percentage of null values imputed: 60.00%
Remaining null values: 8554519


In [44]:
#cpmare table sizes for any anomaly
print(userItemMatrix_imputed.shape[1])
print(userItemMatrix.shape[1])

3706
3706


In [45]:
print(len(ratings))

1000209


In [46]:
ratings_series = userItemMatrix_imputed.stack()
ratings_imputed = ratings_series.reset_index()
ratings_imputed.columns = ['user_id', 'movie_id', 'rating']

In [47]:
print(len(ratings_imputed))

13829721


In [48]:
#Testing for new values
userItemMatrix_imputed

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.00,NaN,0.75,NaN,0.75,NaN,0.75,0.75,NaN,NaN,...,0.75,NaN,0.75,NaN,NaN,0.75,NaN,0.75,NaN,NaN
2,0.75,NaN,NaN,NaN,NaN,0.75,0.75,NaN,0.75,0.75,...,0.75,0.75,NaN,0.75,NaN,NaN,0.75,0.75,0.75,0.75
3,NaN,0.75,NaN,0.75,0.75,NaN,NaN,0.75,NaN,0.75,...,0.75,0.75,NaN,0.75,0.75,NaN,0.75,NaN,0.75,NaN
4,1.00,1.00,NaN,1.00,NaN,1.00,1.00,NaN,NaN,1.00,...,1.00,NaN,1.00,NaN,1.00,NaN,NaN,1.00,1.00,1.00
5,NaN,0.50,0.50,0.50,0.50,0.25,0.50,0.50,0.50,0.50,...,0.50,NaN,0.50,NaN,0.50,0.50,0.50,0.50,0.50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.50,NaN,NaN,0.25,0.50,0.50,NaN,NaN,0.50,0.50,...,NaN,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,NaN
6037,NaN,0.75,NaN,NaN,0.75,0.75,NaN,NaN,NaN,0.75,...,0.75,0.75,0.75,NaN,0.75,0.75,0.75,0.75,0.75,0.75
6038,0.75,0.75,NaN,0.75,NaN,0.75,0.75,0.75,0.75,0.75,...,0.75,NaN,0.75,0.75,0.75,0.75,0.75,0.75,NaN,0.75


In [49]:
user_input_percent = 0.3
subset_percent = 0.7

num_ratings = len(ratings_imputed)
num_user_input = int(num_ratings * user_input_percent)
num_subset = int(num_ratings * subset_percent)

user_input_ratings = ratings_imputed.sample(n=num_user_input, replace=True, random_state=42)
subset_ratings = ratings_imputed.drop(user_input_ratings.index)
user_input_ratings

,user_id,movie_id,rating
6423388,2808,269,0.75
6550634,2863,3284,0.75
10081351,4403,211,0.75
4304572,1880,1089,0.75
13315092,5815,2570,0.75
...,...,...,...
10560935,4611,2511,0.50
7825408,3420,2437,1.00
5894272,2575,2094,0.75
9302140,4063,45,0.75


Imputation portion ends here

In [50]:
# movies['title'] = movies['title'].str.replace(r'\(\d\d\d\d\)', '')
# movies['title'] = movies['title'].apply(lambda x: x.strip())
# movies.head()

In [51]:
# userInput = [
#             {'title':'Breakfast Club, The', 'rating':5},
#             {'title':'Toy Story', 'rating':3.5},
#             {'title':'Jumanji', 'rating':2},
#             {'title':"Pulp Fiction", 'rating':5},
#             {'title':'Akira', 'rating':4.5}
#          ]
# inputMovies = pd.DataFrame(userInput)
# inputMovies

In [52]:
# # inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
# # inputMovies = pd.merge(inputId, inputMovies)
# inputMovies = inputMovies.drop('release_date', 1)
# inputMovies

In [53]:
userSubset = subset_ratings[subset_ratings['movie_id'].isin(user_input_ratings['movie_id'].tolist())]
userSubset.head()

,user_id,movie_id,rating
0,1,1,1.00
2,1,5,0.75
3,1,7,0.75
5,1,11,0.75
6,1,12,0.75


In [54]:
userSubsetGroup = userSubset.groupby(['user_id'])
userSubsetGroup.get_group(222)
mu_k=userSubsetGroup['rating'].mean()

In [55]:
userSubsetGroup=sorted(userSubsetGroup,key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

<ipython-input-55-5489e10f2460>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup=sorted(userSubsetGroup,key=lambda x: len(x[1]), reverse=True)


[(4169,
           user_id  movie_id  rating
  9544701     4169         2    0.50
  9544702     4169         3    0.25
  9544705     4169         6    0.75
  9544706     4169         7    0.75
  9544707     4169        10    0.75
  ...          ...       ...     ...
  9547826     4169      3944    0.75
  9547828     4169      3946    0.00
  9547829     4169      3947    0.75
  9547831     4169      3949    1.00
  9547833     4169      3952    0.75
  
  [2357 rows x 3 columns]),
 (1680,
           user_id  movie_id  rating
  3844502     1680         1    0.75
  3844503     1680         2    0.75
  3844506     1680         6    0.00
  3844507     1680         7    0.50
  3844508     1680         9    0.75
  ...          ...       ...     ...
  3847464     1680      3943    0.75
  3847465     1680      3946    0.75
  3847466     1680      3948    0.75
  3847467     1680      3951    0.75
  3847468     1680      3952    0.75
  
  [2220 rows x 3 columns]),
 (4277,
           user_id  movie_

In [56]:
userSubsetGroup=userSubsetGroup[0:500]

In [57]:
ratings_median=ratings['rating'].median()
print(ratings_median)

0.75


In [58]:
def agreement(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    above_median1 = rating1 > rating_median
    below_median1 = rating1 < rating_median
    above_median2 = rating2 > rating_median
    below_median2 = rating2 < rating_median

    agreement_condition1 = np.logical_and(above_median1, below_median2)
    agreement_condition2 = np.logical_and(below_median1, above_median2)

    return np.logical_or(agreement_condition1, agreement_condition2).any()

def proximity(rating1, rating2, rating_max, rating_min, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    agree = agreement(rating1, rating2, rating_median)
    dist = np.where(agree, np.absolute(rating1 - rating2), 2 * np.absolute(rating1 - rating2))
    prox = ((2 * (rating_max - rating_min) + 1) - dist) ** 2
    return prox

def impact(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    impact_score = (np.absolute(rating1 - rating_median) + 1) * (np.absolute(rating2 - rating_median) + 1)
    agree = agreement(rating1, rating2, rating_median)
    impact_result = np.where(agree, impact_score, 1 / impact_score)
    return impact_result

def popularity(rating1, rating2, mu_k):
    pop = np.where(
        (rating1 > mu_k) & (rating2 > mu_k) | (rating1 < mu_k) & (rating2 < mu_k),
        1 + ((rating1 + rating2) / 2 - mu_k) ** 2,
        1
    )
    return pop

In [59]:
import math
def agreement(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    above_median1 = rating1 > rating_median
    below_median1 = rating1 < rating_median
    above_median2 = rating2 > rating_median
    below_median2 = rating2 < rating_median

    agreement_condition1 = np.logical_and(above_median1, below_median2)
    agreement_condition2 = np.logical_and(below_median1, above_median2)

    return np.logical_or(agreement_condition1, agreement_condition2).any()

def proximity(rating1, rating2, rating_max, rating_min, rating_median):
    D = np.abs(rating1 - rating2)
    med_plus = (rating_max + rating_median) / 2
    med_minus = (rating_min + rating_median) / 2
    Rmax_minus_Rmin_squared = (rating_max - rating_min) ** 2

    agree = agreement(rating1, rating2, rating_median)
    prox = (D - med_plus + med_minus) ** 2 / Rmax_minus_Rmin_squared
    delta = 0.75 if np.abs(rating1 - rating2) > rating_median else (0.5 if np.abs(rating1 - rating2) == rating_median else 0.25)
    return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))

def impact(rating1, rating2, rating_median):
    agree = agreement(rating1, rating2, rating_median)
    impact1= math.exp(-1 / ((np.abs(rating1 - rating_median) + 1) * (np.abs(rating2 - rating_median) + 1)))
    impact2= math.exp(1 / ((np.abs(rating1 - rating_median) + 1) * (np.abs(rating2 - rating_median) + 1)))
    return impact1 if agree else impact2

def proposed_popularity(rating1, rating2,rIi):
    if (rating1 > rIi and rating2 > rIi) or (rating1 < rIi and rating2 < rIi):
        return math.log10(2 + (((rating1 + rating2) / 2) - rIi) ** 2)
    else:
        return 0.3010


In [60]:
import numpy as np
import pandas as pd

def pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, rating_median, mu_k):
    pip_similarity_sum = 0
    n_ratings = 0

    for rating1, rating2 in zip(tempRatingList, tempGroupList):
        if not np.isnan(rating1) and not np.isnan(rating2):
            prox = proximity(rating1, rating2, rating_max, rating_min, rating_median)
            imp = impact(rating1, rating2, rating_median)
            pop = popularity(rating1, rating2,rating_median )

            pip_similarity_sum += prox * imp * pop
            n_ratings += 1

    if n_ratings == 0:
        return 0

    pip_similarity_value = pip_similarity_sum / n_ratings
    return pip_similarity_value

PIPdict = {}

rating_max = 5
rating_min = 0

for name, group in userSubsetGroup:

    group = group.sort_values(by='movie_id')
    inputMovies = user_input_ratings.sort_values(by='movie_id')
    temp_df = inputMovies[inputMovies['movie_id'].isin(group['movie_id'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    pip_similarity_value = pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, ratings_median, mu_k)
    PIPdict[name] = pip_similarity_value.mean()

#print(sorted(PIPdict.keys()))
print(PIPdict[48])

<ipython-input-59-d47755b63b66>:25: RuntimeWarning: divide by zero encountered in double_scalars
  return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))


inf


In [61]:
import numpy as np
import pandas as pd

def pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, rating_median, mu_k):
    pip_similarity_sum = 0
    n_ratings = 0

    for rating1, rating2 in zip(tempRatingList, tempGroupList):
        if not np.isnan(rating1) and not np.isnan(rating2):
            prox = proximity(rating1, rating2, rating_max, rating_min, rating_median)
            imp = impact(rating1, rating2, rating_median)
            pop = popularity(rating1, rating2,rating_median )

            pip_similarity_sum += prox * imp * pop
            n_ratings += 1

    if n_ratings == 0:
        return 0

    pip_similarity_value = pip_similarity_sum / n_ratings
    return pip_similarity_value

PIPdict = {}

rating_max = 5
rating_min = 0

for name, group in userSubsetGroup:
  group = group.sort_values(by='movie_id')
  inputMovies = user_input_ratings.sort_values(by='movie_id')
  temp_df = inputMovies[inputMovies['movie_id'].isin(group['movie_id'].tolist())]
  tempRatingList = temp_df['rating'].tolist()
  tempGroupList = group['rating'].tolist()

  similarity_values = []

  for temp_rating, group_rating in zip(tempRatingList, tempGroupList):
      pip_similarity_value = pip_similarity([temp_rating], [group_rating], rating_max, rating_min, ratings_median, mu_k)
      similarity_values.append(pip_similarity_value)

print(similarity_values)

<ipython-input-59-d47755b63b66>:25: RuntimeWarning: divide by zero encountered in double_scalars
  return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))


[inf, inf, 0.08902163713969871, 0.08902163713969871, inf, 0.08902163713969871, 0.08902163713969871, inf, inf, inf, inf, 0.08902163713969871, 0.08902163713969871, inf, inf, inf, inf, inf, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, inf, inf, 0.08902163713969871, 0.19459403634297662, 0.08902163713969871, inf, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, inf, inf, 0.08902163713969871, inf, inf, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, inf, 0.03895468082109352, 0.08902163713969871, inf, inf, inf, inf, inf, 0.08902163713969871, inf, 0.08902163713969871, inf, 0.08902163713969871, inf, inf, inf, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, inf, 0.08902163713969871, inf, 0.08902163713969871, inf, 0.08902163713969871, inf, 0.08902163713969871, 0.08902163713969871, 0.08902163713969871, 0.08902163

In [62]:
#PIP_DF = pd.DataFrame.from_dict(PIPdict, orient='index')
PIP_DF = pd.DataFrame(similarity_values, columns=['Similarity'])
PIP_DF.columns = ['similarityIndex']
PIP_DF['userId'] = PIP_DF.index
PIP_DF.index = range(len(PIP_DF))

#removing the inf values

PIP_DF['similarityIndex'] = PIP_DF['similarityIndex'].replace([np.inf, -np.inf], 0)
PIP_DF.head()

,similarityIndex,userId
0,0.000000,0
1,0.000000,1
2,0.089022,2
3,0.089022,3
4,0.000000,4


In [63]:
topUsers=PIP_DF.sort_values(by='similarityIndex', ascending=False)
topUsers.head(100)

,similarityIndex,userId
636,0.194594,636
1621,0.194594,1621
583,0.194594,583
478,0.194594,478
1243,0.194594,1243
...,...,...
975,0.089022,975
1052,0.089022,1052
1057,0.089022,1057
1061,0.089022,1061


In [64]:
topUsersRating=topUsers.merge(ratings, left_on='userId', right_on='user_id', how='inner')
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
#topUsersRating.head(18000)
topUsersRating.head(1352)
topUsersRating.isnull().sum()

similarityIndex    0
userId             0
user_id            0
movie_id           0
rating             0
weightedRating     0
dtype: int64

In [65]:
tempTopUsersRating = topUsersRating.groupby('movie_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
#handled the zero division error bug
recommendation_df.loc[tempTopUsersRating['sum_similarityIndex'] == 0, 'weighted average recommendation score'] = 0
recommendation_df.fillna(0, inplace=True)
##
recommendation_df['movieID'] = tempTopUsersRating.index
recommendation_df.tail()

,weighted average recommendation score,movieID
movie_id,,
3948,0.684407,3948
3949,0.859066,3949
3950,0.638240,3950
3951,0.789289,3951
3952,0.677951,3952


In [66]:
tempTopUsersRating.head(973)

,sum_similarityIndex,sum_weightedRating
movie_id,,
1,27.195684,21.328126
2,9.612782,5.185553
3,8.369209,3.795022
4,1.958332,0.747137
5,4.412347,2.129634
...,...,...
1084,9.075833,7.014401
1085,1.141514,0.868652
1086,4.206006,3.128112


In [67]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(100)

,weighted average recommendation score,movieID
movie_id,,
787,1.000000,787
2830,1.000000,2830
83,1.000000,83
1002,1.000000,1002
682,1.000000,682
...,...,...
3038,0.860839,3038
3435,0.859851,3435
326,0.859714,326


In [68]:
recommendation_df = recommendation_df[(recommendation_df['weighted average recommendation score'] <= 5) &
 (recommendation_df['weighted average recommendation score'] >= 0) &
 (recommendation_df['weighted average recommendation score'].notnull())]
print(recommendation_df)

          weighted average recommendation score  movieID
movie_id                                                
787                                         1.0      787
2830                                        1.0     2830
83                                          1.0       83
1002                                        1.0     1002
682                                         1.0      682
...                                         ...      ...
2246                                        0.0     2246
1436                                        0.0     1436
1433                                        0.0     1433
2277                                        0.0     2277
3312                                        0.0     3312

[3569 rows x 2 columns]


In [69]:
print(recommendation_df.isnull().sum())

weighted average recommendation score    0
movieID                                  0
dtype: int64


In [70]:
recommendation_df.head(10)

,weighted average recommendation score,movieID
movie_id,,
787,1.0,787
2830,1.0,2830
83,1.0,83
1002,1.0,1002
682,1.0,682
2197,1.0,2197
1796,1.0,1796
2984,1.0,2984
2824,1.0,2824


In [71]:
movies.loc[movies['movie_id'].isin(recommendation_df.head(10)['movieID'].tolist())]

,movie_id,title
82,83,Once Upon a Time... When We Were Colored (1995)
675,682,Tigrero: A Film That Was Never Made (1994)
777,787,"Gate of Heavenly Peace, The (1995)"
989,1002,Ed's Next Move (1996)
1736,1796,In God's Hands (1998)
2128,2197,Firelight (1997)
2755,2824,On the Ropes (1999)
2761,2830,Cabaret Balkan (Bure Baruta) (1998)
2915,2984,On Any Sunday (1971)
2930,2999,Man of the Century (1999)


In [72]:
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [73]:
from math import sqrt
merged_ratings = pd.merge(recommendation_df, ratings,on='movie_id')
merged_ratings['squared_diff'] = (merged_ratings['weighted average recommendation score'] - merged_ratings['rating']) ** 2
mean_squared_diff = merged_ratings['squared_diff'].mean()
mean_squared_diff
rmse_score = sqrt(mean_squared_diff)
print("RMSE Score:", rmse_score)

RMSE Score: 0.24701935120198082


In [74]:
import pandas as pd
from math import sqrt
merged_ratings = pd.merge(recommendation_df, ratings, on='movie_id')

merged_ratings['abs_diff'] = abs(merged_ratings['weighted average recommendation score'] - merged_ratings['rating'])

mae_score = merged_ratings['abs_diff'].mean()

print("MAE Score:", mae_score)

MAE Score: 0.19629675344615538
